# # NetList Format if text file

Primary Output<br>
Circuit Name<br>
Primary Input<br>
OUT_NODE GATE_TYPE IN_NODE<br>
....<br>
END<br>
Gate type can be:<br>
AND<br>
OR<br>
NOT<br>
XOR<br>

### Else 
You can edit the netlist give below in the format
["Primary Input", "Circuit Name","Primary Output","OUT_NODE GATE_TYPE IN_NODE", .... ,"END"]


In [1]:
#This function load text file and create the NETLIST

def netlist_loader(path):
    f = open(path, "r")
    net = f.readlines()
    netlist = []
    for i in net:
        netlist.append(i.strip())
    return netlist

In [2]:
def input_file_generator(path):
    f = open(path, "r")
    input_list_preprocessing = f.readlines()
    input_list = []    
    for i in input_list_preprocessing:
        input_list.append(i.strip().split(" "))
    return input_list 

In [3]:
#Netlist format if you don't want to input a text file
netlist = ["H","CKT_NAME","A B C D X","E AND A B","G AND C D","F OR E X","H OR G F","END"]

In [4]:
#My circuit
class circuit:
    
    def __init__(self,netlist):
        self.netlist = netlist
        #Primary outputs nodes
        self.PO = netlist[0].split(" ")
        self.CKT_NAME = netlist[1]
        
        #Primary Input nodes
        self.PI = netlist[2].split(" ")
        
        #Forming array of all gates in the ciruit
        self.GATES_ARRAY = self.make_gate_list()
        
        #Layer wise output is the ouput nodes available after each layer
        #Layer wise gate is the index of gate after each layer
        #Input Available will be used to map the value of each node to calculated
        print("reached here")
        self.layer_wise_output,self.layer_wise_gate,self.input_available =self.layer_circuit()
    
    def make_gate_list(self):
        #Makes the array of gate from the netlist
        GATES_ARRAY = []

        for i in range(3,len(self.netlist)):
            if self.netlist[i] =="END":
                break
            gate_specs = self.netlist[i].split(" ")
            gate_output = [gate_specs[0]]
            gate_type = gate_specs[1]
            gate_input = []
    
            for j in range(len(gate_specs)-2):
                gate_input.append(gate_specs[j+2])
            GATES_ARRAY.append([gate_output,gate_type,gate_input,0])
            
        #Output format is [[[output node],"Gate Type",[Input Node],0],[],...]
        #0 as the last elemet in this list to show it has not yet been placed on the flow graph
        return GATES_ARRAY
    
    def layer_circuit(self):
        # Helps to divide circuit in layer for execution
        #Layer wise output is the ouput nodes available after each layer
        layer_wise_output = []
        
        #Layer wise gate is the index of gate after each layer
        layer_wise_gate =[]
        
        #Input available keeps track of node available after layers
        input_available = []
        
        #Primary Input available before execution
        input_available += self.PI
        
        #Indicator to show all output nodes are available and the flow graph of circuit has been formed
        layer_not_completed = True

        while layer_not_completed:
            
            #ouput nodes available after this layer
            curr_layer = []
            
            #index of gate performed in this layer 
            curr_layer_gate = []
            
            #new input that next layer can use from this layer output
            input_available_next = []
            
            for i,gate in enumerate(self.GATES_ARRAY):
                
                #Check if gate already used if so go to next gate
                if gate[3] == 1:
                    
                    continue
                    
                gate_ready = True
                
                #Check if all input are available
                for j in gate[2]:
                    if j not in input_available:
                        #Input not availabe so move to next gate
                        gate_ready = False
                        break 
                        
                #If gate can be used, add its output to inpu available list and index and output node to their respective list
                if gate_ready:
                    input_available_next += gate[0]
                    curr_layer+=gate[0]
                    curr_layer_gate += [i]
                    gate[3] = 1
                    
            input_available += input_available_next             
            layer_wise_output.append(curr_layer)
            layer_wise_gate.append(curr_layer_gate)
            
            #Check if all primary outputs are available or not
            for i in self.PO:
                if i not in input_available:
                    break;
                elif i == self.PO[-1]:
                    layer_not_completed = False
        return layer_wise_output, layer_wise_gate,input_available
    
    
    def gate_ouput_calculation(self,g_type,g_input,value):
    
    #Gives output of the current gate
    
        if g_type == "AND":
            for i in g_input:
                if value[i] == 0:
                    return 0
            return 1
        
        if g_type == "OR":
            for i in g_input:
                if value[i] == 1:
                    return 1
            return 0
    
        if g_type == "NOT":
            for i in g_input:
                if value[i] == 1:
                    return 0
                else:
                    return 1
        if g_type == "XOR":
            sum_v = 0
            for i in g_input:
                sum_v += value[i]
            return sum_v%2
                
    def input_applied(self,input_list):
        nodes_value = {}
        for i in self.input_available:
            nodes_value[i] = -1
        for i in input_list:
            nodes_value[i[0]] = i[1]
        
        
        for layer_gate in self.layer_wise_gate:
            for j in layer_gate:
                gate_type = self.GATES_ARRAY[j][1]
                gate_output_node = self.GATES_ARRAY[j][0][0]
                gate_value = self.gate_ouput_calculation(gate_type,self.GATES_ARRAY[j][2],nodes_value)
                nodes_value[gate_output_node] = gate_value
        return nodes_value
    
    def output(self,inp):
        input_list = []
        for i in inp:
            value = i.split(" ")
            input_list.append([value[0],int(value[1])])
        node_value = self.input_applied(input_list)
        for i in self.PO:
            print(i," = ",node_value[i])
        print(node_value)
            
    def outputs(self,inps):
        for inp in inps:
            input_list = []
            
            for i,inpt in enumerate(inp):
                
                input_list.append([self.PI[i],int(inpt)])
                print(self.PI[i], "=", int(inpt),end = " ")
            node_value = self.input_applied(input_list)
            for i in self.PO:
                print(i," = ",node_value[i],end = " ")
            print("\n",end="")

In [5]:
ckta = circuit(netlist)

reached here


In [6]:
inp = ["A 1","B 0","C 1","D 0", "X 1"]

In [7]:
ckta.output(inp)

H  =  1
{'A': 1, 'B': 0, 'C': 1, 'D': 0, 'X': 1, 'E': 0, 'G': 0, 'F': 1, 'H': 1}


In [8]:
path = r"C:\Users\vipin\Desktop\Semester 5\EE 677\Assignment 1\Netlist.txt"

In [9]:
netlist = netlist_loader(path)

In [10]:
netlist

['X', 'CKT_NAME', 'A B C D', 'E AND A B', 'F AND C D', 'X OR E F', 'END']

In [11]:
cktb = circuit(netlist)

reached here


In [12]:
cktb.output(inp)

X  =  0
{'A': 1, 'B': 0, 'C': 1, 'D': 0, 'E': 0, 'F': 0, 'X': 0}


## Giving input as list of array with input format same as the PI input format

In [13]:
input_list = [["0", "0", "0", "0"],["0", "1", "0", "0"],["0", "0", "0", "1"],["1","1","0","0"]]

In [14]:
cktb.outputs(input_list)

A = 0 B = 0 C = 0 D = 0 X  =  0 
A = 0 B = 1 C = 0 D = 0 X  =  0 
A = 0 B = 0 C = 0 D = 1 X  =  0 
A = 1 B = 1 C = 0 D = 0 X  =  1 


In [15]:
input_file_path = r"C:\Users\vipin\Desktop\Semester 5\EE 677\Assignment 1\Input.txt"

In [16]:
input_file = input_file_generator(input_file_path)

In [17]:
cktb.outputs(input_file)

A = 0 B = 0 C = 0 D = 0 X  =  0 
A = 0 B = 0 C = 0 D = 1 X  =  0 
A = 0 B = 0 C = 1 D = 0 X  =  0 
A = 0 B = 0 C = 1 D = 1 X  =  1 
A = 0 B = 1 C = 0 D = 0 X  =  0 
A = 0 B = 1 C = 0 D = 1 X  =  0 
A = 0 B = 1 C = 1 D = 0 X  =  0 
A = 0 B = 1 C = 1 D = 1 X  =  1 
A = 1 B = 0 C = 0 D = 0 X  =  0 
A = 1 B = 0 C = 0 D = 1 X  =  0 
A = 1 B = 0 C = 1 D = 0 X  =  0 
A = 1 B = 0 C = 1 D = 1 X  =  1 
A = 1 B = 1 C = 0 D = 0 X  =  1 
A = 1 B = 1 C = 0 D = 1 X  =  1 
A = 1 B = 1 C = 1 D = 0 X  =  1 
A = 1 B = 1 C = 1 D = 1 X  =  1 
